# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df=pd.read_csv("../output_data/cities.csv")
cities_df

,city,lat_lngs,lat,long,temp,max_temp,humidity,wind_speed,clouds
0,sitka,"(54.195017882651655, -140.9961652832942)",54.195018,-140.996165,68.00,68.00,64,4.70,1
1,sao joao da barra,"(-32.56067629162232, -24.477583095776794)",-32.560676,-24.477583,74.80,74.80,92,5.79,100
2,gizo,"(-10.657827351856753, 157.44736886399937)",-10.657827,157.447369,84.00,84.00,75,12.03,7
3,vila franca do campo,"(37.0216709273888, -21.829063954293986)",37.021671,-21.829064,70.00,70.00,72,1.01,0
4,butaritari,"(21.46527150153588, 163.18601932759464)",21.465272,163.186019,82.94,82.94,73,16.26,0
...,...,...,...,...,...,...,...,...,...
548,prince albert,"(53.37409289538326, -105.94598620485037)",53.374093,-105.945986,84.20,84.20,62,6.93,1
549,reinosa,"(42.97526332330935, -3.7740878513133964)",42.975263,-3.774088,69.24,69.80,83,4.70,90
550,buenaventura,"(4.1072731914195515, -76.84195954906856)",4.107273,-76.841960,76.30,76.30,92,1.83,95
551,ossora,"(59.57454242406632, 162.47252962179505)",59.574542,162.472530,58.57,58.57,85,7.85,94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations_df = cities_df[["lat", "long"]].astype(float)

humidity_df = cities_df["humidity"].astype(float)

In [5]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_temp = pd.DataFrame()

for index,row in cities_df.iterrows():
    if row["max_temp"]>75 and row["max_temp"]<100 and row["wind_speed"]<5 and row["clouds"]==0:
        ideal_temp[index]=row
ideal_temp.dropna().T

,city,lat_lngs,lat,long,temp,max_temp,humidity,wind_speed,clouds
159,taoudenni,"(20.708368584826346, -7.673734428750549)",20.7084,-7.67373,97,97,27,3.22,0
190,marsa matruh,"(31.327298940513643, 28.351484968873848)",31.3273,28.3515,77,77,69,2.24,0
194,bolgatanga,"(10.82128577819698, -0.6538189019736649)",10.8213,-0.653819,75.99,75.99,83,3.74,0
241,naples,"(25.34896993018127, -82.34369293086765)",25.349,-82.3437,76.57,78.01,61,1.12,0
248,caserta,"(41.235227772080094, 14.423785452911488)",41.2352,14.4238,76.17,78.01,61,1.12,0
264,tabas,"(34.52071173748688, 55.30941695517893)",34.5207,55.3094,87.8,87.8,20,2.24,0
329,tigzirt,"(36.95521192617227, 3.6937020265041838)",36.9552,3.6937,82.65,82.65,41,2.01,0
337,tiznit,"(29.00826629203867, -10.393622379847244)",29.0083,-10.3936,75.72,75.72,45,0.96,0
358,iwanai,"(43.656072944236286, 139.62617462036104)",43.6561,139.626,80.1,80.1,51,2.71,0
362,bawku,"(11.029625128618079, -0.5432972279132287)",11.0296,-0.543297,76.75,76.75,80,4.54,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotels_df=ideal_temp.loc[["city","country","lat","long"]].T
hotels_df["Hotel"]=""
hotels_df["Hotel Lat"]=""
hotels_df["Hotel Lng"]=""
hotels_df["Rating"]=""
hotels_df["Reviews"]=""
hotels_df.head()

D:\Users\zanny\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,city,country,lat,long,Hotel,Hotel Lat,Hotel Lng,Rating,Reviews
159,taoudenni,NaN,20.7084,-7.67373,,,,,
190,marsa matruh,NaN,31.3273,28.3515,,,,,
194,bolgatanga,NaN,10.8213,-0.653819,,,,,
241,naples,NaN,25.349,-82.3437,,,,,
248,caserta,NaN,41.2352,14.4238,,,,,


In [12]:
nearby_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index,row in hotels_df.iterrows():
    params = {
    "location": " {} , {} ".format(row["lat"],row["long"]) ,
    "rankby": "distance",
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key}
    response = requests.get(nearby_url, params=params).json()
    
    hotels_df.loc[index,"Hotel Lat"]=response["results"][0]['geometry']['location']["lat"]
    hotels_df.loc[index,"Hotel Lng"]=response["results"][0]['geometry']['location']['long']
    hotels_df.loc[index,"Hotel"]=response["results"][0]["name"]
    hotels_df.loc[index,"Rating"]=response["results"][0]["rating"]
    hotels_df.loc[index,"Reviews"]=response["results"][0]["user_ratings_total"]

hotels_df

IndexError: list index out of range

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

KeyError: 'City'

In [14]:
# Add marker layer ontop of heat map
hotel_loc=[]
for index,row in hotels_df.iterrows():
    hotel_loc.append( tuple([row["Hotel Lat"],row["Hotel Lng"]] )) 
markers=gmaps.marker_layer(hotel_loc)
fig.add_layer(markers)


# Display Map
fig

TraitError: Element of the 'location' trait of a Marker instance must be a valid latitude (-90 <= latitude <= 90), but a value of '' <class 'str'> was specified.